In [1]:
# <수행 방법 요약>
# 이미지에서 각 모델들이 인식한 객체들에게서 바운딩된 박스, 신뢰도, 라벨들을 추출한다.
# 모든 박스를 서로 한 쌍씩 겹침 정도, 즉 iou를 계산한다.
# iou를 1에서 뺀 값이 특정 수치보다 작으면 많이 겹쳤다고 판단한다.
# 서로 많이 겹쳐진 박스들끼리 그룹화를 한다.
# 각 그룹에서 박스들의 좌표의 평균을 최종 박스 좌표로 결정한다.
# 각 그룹에서 박스들의 신뢰도의 평균을 최종 신뢰도로 결정한다.
# 각 그룹에서 박스들의 라벨 이름중 빈도수가 높은것을 최종 라벨 이름으로 결정한다.(다수결)
# 이미지에 최종 박스 좌표 위치에 사각형을 그리고, 신뢰도와 라벨 이름을 텍스트로 표시한다.

from ultralytics import YOLO
import numpy as np
from scipy.spatial.distance import cdist
from collections import Counter, defaultdict
import cv2

# 모델 파일명 리스트
model_files = [f'models/model{i}.pt' for i in range(1, 13)]

# 모델 로드 및 names 저장
models = []
model_names = {}
class_name_mapping = {}

# 모든 모델의 클래스 통합
for i, model_file in enumerate(model_files):
    model = YOLO(model_file)
    models.append(model)
    model_names[f'model{i+1}'] = model.names
    for class_id, class_name in model.names.items():
        if class_name not in class_name_mapping:
            class_name_mapping[class_name] = []
        class_name_mapping[class_name].append((f'model{i+1}', class_id))

# 모든 모델 예측 수행, 결과 담기
def ensemble_predict(image):
    results = []
    for model in models:
        results.append(model.predict(image, conf=0.5))
    combined_results = combine_results(*results) # final_boxes, final_confidences, final_labels
    return combined_results

# 각 결과에서 바운딩 박스, 신뢰도, 라벨 추출
def combine_results(*results):
    combined_boxes = []
    combined_confidences = []
    combined_labels = defaultdict(list)  # 같은 박스 위치에 여러 레이블을 저장하기 위한 딕셔너리

    # 각 모델의 결과에서 바운딩 박스를 추출하여 결합
    for model_index, result_list in enumerate(results):
        model_name = f'model{model_index + 1}'
        for result in result_list:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())
                
                # 클래스 번호를 클래스 이름으로 변환
                class_name = model_names[model_name][class_id]
                
                combined_boxes.append([x1, y1, x2, y2])
                combined_confidences.append(conf)
                combined_labels[(x1, y1, x2, y2)].append(class_name)
    # print('combined_boxes:')
    # print(combined_boxes)
                
    # print('combined_confidences:')
    # print(combined_confidences)
                
    # print('combined_labels:')
    # print(combined_labels)

    combined_boxes = np.array(combined_boxes)
    combined_confidences = np.array(combined_confidences)
    
    # 박스를 그룹화하여 다수결로 라벨 결정
    final_boxes = []
    final_confidences = []
    final_labels = []

    box_groups = group_boxes_by_overlap(combined_boxes) # 겹치는 박스 번호들끼리 그룹화한 리스트
    # print('box_groups:')
    # print(box_groups)

    for group in box_groups:
        # print('group:')
        # print(group)
        group_boxes = combined_boxes[group] # 그룹의 각 번호에 해당하는 박스들의 좌표배열
        # print('group_boxes:')
        # print(group_boxes)
        group_confidences = combined_confidences[group] # 그룹의 각 번호에 해당하는 박스들의 신뢰도
        group_labels = [combined_labels[tuple(box)] for box in group_boxes] # combined_labels = {(박스 좌표배열) : 라벨이름}
                                                                            # 키값으로 조회한 밸류를 저장함
        # print('group_labels:')
        # print(group_labels)

        # 그룹 내에서 평균 박스와 평균 신뢰도를 계산
        avg_box = np.mean(group_boxes, axis=0)
        avg_conf = np.mean(group_confidences)
        
        # 각 박스에 대해 다수결로 레이블 결정
        flattened_labels = [label for sublist in group_labels for label in sublist] # 라벨이름들이 저장된 리스트
        # print('flattened_labels:')
        # print(flattened_labels)
        most_common_label = Counter(flattened_labels).most_common(1)[0][0] # 가장 빈도수가 높은 라벨 추출
        # print('Counter(flattened_labels):')
        # print(Counter(flattened_labels))
        # print('Counter(flattened_labels).most_common(1):')
        # print(Counter(flattened_labels).most_common(1))

        final_boxes.append(avg_box)
        final_confidences.append(avg_conf)
        final_labels.append(most_common_label)

    final_boxes = np.array(final_boxes)
    final_confidences = np.array(final_confidences)
    final_labels = np.array(final_labels)

    return final_boxes, final_confidences, final_labels

# 박스가 겹치는 그룹을 찾는 함수(각 그룹 안에는 박스 번호들이 있음)
def group_boxes_by_overlap(boxes, iou_threshold=0.4):
    # print('boxes:')
    # print(boxes)
    distances = cdist(boxes, boxes, lambda x, y: 1 - iou(x, y)) # 두 박스간의 겹침 정도(비율)가 클수록 1에서 뺀 값이 작아지므로 거리가 작아진다.
    # print('distances:')
    # print(distances)
    groups = []
    visited = set()

    for i in range(len(boxes)): # i: 현재 박스의 인덱스
        if i in visited: # 이미 방문된 박스 집합(visited)에 있다면 다음 반복으로 넘어감
            continue
        group = [i] # 현재 박스 번호를 포함한 그룹 생성
        visited.add(i) # 현재 박스 번호를 집합에 추가
        for j in range(i + 1, len(boxes)): # 현재 박스의 다음 박스 번호부터 순회
            if j in visited: # 이미 방문된 박스 집합(visited)에 있다면 다음 반복으로 넘어감
                continue
            if distances[i, j] < iou_threshold: # 박스 i와 j의 거리가 iou_threshold보다 작다면, 두 박스가 많이 겹친다고 판단함
                group.append(j) # 박스 번호 j를 group에 추가
                visited.add(j) # 집합에도 추가
        groups.append(group) # 그룹을 groups에 추가
    return groups

# IoU (Intersection over Union) 계산 함수
# 객체 탐지 및 이미지 분석에서 두 바운딩 박스 간의 겹침 정도를 측정하는 지표,
# IoU는 두 박스의 교차 영역과 두 박스의 합집합 영역 간의 비율을 나타냄
def iou(box1, box2):
    # print('box1:', box1)
    # print('box2:', box2)
    # print('==' * 50)
    x1 = max(box1[0], box2[0]) # x1 vs X1, 더 큰 값이 교집합 영역 좌상단 x좌표
    y1 = max(box1[1], box2[1]) # y1 vs Y1, 더 큰 값이 교집합 영역 좌상단 y좌표
    x2 = min(box1[2], box2[2]) # x2 vs X2, 더 작은 값이 교집합 영역 우하단 x좌표
    y2 = min(box1[3], box2[3]) # y2 vs Y2, 더 작은 값이 교집합 영역 우하단 y좌표

    intersection = max(0, x2 - x1) * max(0, y2 - y1) # 교집합 영역 가로길이 * 교집합 영역 세로길이 = 교집합 영역의 전체 면적
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1]) # 박스1의 가로길이 * 세로길이 = 박스1의 전체 면적
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1]) # 박스2의 가로길이 * 세로길이 = 박스2의 전체 면적
    union = box1_area + box2_area - intersection # 두 박스의 합집합 영역에서 교집합 영역을 뺀 값

    return intersection / union if union > 0 else 0 # 교집합 면적을 합집합 면적으로 나눈값

# 이미지 파일을 읽어오기
image_path = 'potato.jpg'
image = cv2.imread(image_path)

# 이미지에 대해 앙상블 예측 수행
combined_results = ensemble_predict(image) # final_boxes, final_confidences, final_labels

# 결과 출력 및 이미지에 그리기
boxes, confidences, labels = combined_results

print("\ndetect:")
for box, conf, label in zip(boxes, confidences, labels):
    x1, y1, x2, y2 = map(int, box)
    label_name = label  # 이미 label은 클래스 이름입니다.

    # 레이블 이름이 확인되었으면, 이미지를 수정합니다.
    if label_name is not None:
        # 사각형 테두리 그리는 함수(사각형을 그릴 이미지, 사각형 좌상단 좌표, 사각형 우하단 좌표, 사각형 색, 사각형 두께)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # 텍스트 추가하는 함수(텍스트 추가할 이미지, 텍스트 문자열, 텍스트 시작점, 텍스트 폰트, 텍스트 크기, 텍스트 색상, 텍스트 두께)
        cv2.putText(image, f'{label_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # 박스 좌표, 신뢰도, 라벨이름 출력
        print(f'\nBox: {box}, Confidence: {conf}, Label: {label_name}')

# 수정된 이미지 저장
output_image_path = 'potato_with_boxes.jpg'
cv2.imwrite(output_image_path, image)
print(f"\nAnnotated image saved as {output_image_path}")



0: 640x640 (no detections), 18.4ms
Speed: 2.0ms preprocess, 18.4ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 20.1ms
Speed: 2.0ms preprocess, 20.1ms inference, 40.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 19.8ms
Speed: 1.0ms preprocess, 19.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 20.1ms
Speed: 1.5ms preprocess, 20.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Eggs, 19.5ms
Speed: 2.0ms preprocess, 19.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 18.9ms
Speed: 1.0ms preprocess, 18.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 19.6ms
Speed: 1.0ms preprocess, 19.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 18.7ms
Speed: 2.0ms preprocess, 18.7ms inference, 0.0ms postprocess per 